In [459]:
import pandas as pd
pd.set_option('display.max_columns', 50000)
import numpy as np
import csv
import json
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

In [460]:
df = pd.read_csv(filepath_or_buffer="./dataset/data.csv")
df.head()

,user_id,username,age,status,sex,orientation,drinks,drugs,height,job,location,pets,smokes,language,new_languages,body_profile,education_level,dropped_out,bio,interests,other_interests,location_preference
0,fffe3100,Edith Lopez,27,single,f,gay,socially,never,66.0,medicine / health,"oakland, california",likes dogs and likes cats,no,"english (fluently), spanish (poorly), sign lan...",interested,athletic,4.0,no,bottom line i love life! i work hard and i lov...,sports,instruments,same state
1,fffe3200,Travis Young,26,single,m,gay,socially,never,68.0,other,"pleasant hill, california",likes dogs,no,"english (fluently), tagalog (okay), french (po...",interested,fit,3.0,no,"i'm a straightforward, genuine, fun loving (i'...",painting,instruments,anywhere
2,fffe3300,Agnes Smith,20,seeing someone,f,bisexual,socially,sometimes,69.0,other,"oakland, california",has dogs and likes cats,sometimes,"english (fluently), sign language (poorly), fr...",interested,fit,2.0,no,mmmmm yummy tacosss. yoga is where it's at. i ...,instruments,dancing,same city
3,fffe3400,Salvador Klaver,27,single,m,bisexual,socially,sometimes,68.0,computer / hardware / software,"daly city, california",likes dogs and likes cats,no,english,not interested,average,3.0,no,i'm a stealth geek. that special mix of techni...,sketching,acting,same city
4,fffe3500,Elana Sewell,22,single,f,bisexual,often,sometimes,68.0,other,"oakland, california",likes dogs and likes cats,yes,english,not interested,average,2.0,yes,with the whisper of the wind i was weaved into...,craft,designing,same city


In [461]:
df.describe()

,age,height,education_level
count,2001.000000,2001.000000,2001.000000
mean,33.072464,68.289855,2.932534
std,10.483189,3.895246,0.812371
min,18.000000,55.000000,1.000000
25%,26.000000,65.000000,2.000000
50%,30.000000,68.000000,3.000000
75%,39.000000,71.000000,3.000000
max,69.000000,83.000000,5.000000


In [462]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2001 entries, 0 to 2000
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   user_id              2001 non-null   object 
 1   username             2001 non-null   object 
 2   age                  2001 non-null   int64  
 3   status               2001 non-null   object 
 4   sex                  2001 non-null   object 
 5   orientation          2001 non-null   object 
 6   drinks               2001 non-null   object 
 7   drugs                2001 non-null   object 
 8   height               2001 non-null   float64
 9   job                  2001 non-null   object 
 10  location             2001 non-null   object 
 11  pets                 2001 non-null   object 
 12  smokes               2001 non-null   object 
 13  language             2001 non-null   object 
 14  new_languages        2001 non-null   object 
 15  body_profile         2001 non-null   o

In [463]:
df["likes_dogs"] = False
df["likes_cats"] = False
df["has_cats"] = False
df["has_dogs"] = False
df["location_state"] = None
df["location_city"] = None
df["language_json"] = None
df.head()

,user_id,username,age,status,sex,orientation,drinks,drugs,height,job,location,pets,smokes,language,new_languages,body_profile,education_level,dropped_out,bio,interests,other_interests,location_preference,likes_dogs,likes_cats,has_cats,has_dogs,location_state,location_city,language_json
0,fffe3100,Edith Lopez,27,single,f,gay,socially,never,66.0,medicine / health,"oakland, california",likes dogs and likes cats,no,"english (fluently), spanish (poorly), sign lan...",interested,athletic,4.0,no,bottom line i love life! i work hard and i lov...,sports,instruments,same state,False,False,False,False,None,None,None
1,fffe3200,Travis Young,26,single,m,gay,socially,never,68.0,other,"pleasant hill, california",likes dogs,no,"english (fluently), tagalog (okay), french (po...",interested,fit,3.0,no,"i'm a straightforward, genuine, fun loving (i'...",painting,instruments,anywhere,False,False,False,False,None,None,None
2,fffe3300,Agnes Smith,20,seeing someone,f,bisexual,socially,sometimes,69.0,other,"oakland, california",has dogs and likes cats,sometimes,"english (fluently), sign language (poorly), fr...",interested,fit,2.0,no,mmmmm yummy tacosss. yoga is where it's at. i ...,instruments,dancing,same city,False,False,False,False,None,None,None
3,fffe3400,Salvador Klaver,27,single,m,bisexual,socially,sometimes,68.0,computer / hardware / software,"daly city, california",likes dogs and likes cats,no,english,not interested,average,3.0,no,i'm a stealth geek. that special mix of techni...,sketching,acting,same city,False,False,False,False,None,None,None
4,fffe3500,Elana Sewell,22,single,f,bisexual,often,sometimes,68.0,other,"oakland, california",likes dogs and likes cats,yes,english,not interested,average,2.0,yes,with the whisper of the wind i was weaved into...,craft,designing,same city,False,False,False,False,None,None,None


In [464]:
print(df["status"].value_counts())
print("-"*50)
print(df["sex"].value_counts())
print("-"*50)
print(df["orientation"].value_counts())
print("-"*50)
print(df["drinks"].value_counts())
print("-"*50)
print(df["drugs"].value_counts())
print("-"*50)
print(df["smokes"].value_counts())
print("-"*50)
print(df["new_languages"].value_counts())
print("-"*50)
print(df["body_profile"].value_counts())
print("-"*50)
print(df["dropped_out"].value_counts())
print("-"*50)
print(df["interests"].value_counts())
print("-"*50)
print(df["other_interests"].value_counts())
print("-"*50)
print(df["location_preference"].value_counts())
print("-"*50)
print(df["location"].value_counts())
print("-"*50)
print(df["pets"].value_counts())
print("-"*50)
print("-"*50)
print("-"*50)
print(df["likes_dogs"].value_counts())
print("-"*50)
print(df["likes_cats"].value_counts())
print("-"*50)
print(df["has_cats"].value_counts())
print("-"*50)
print(df["has_dogs"].value_counts())
print("-"*50)
print(df["location_state"].value_counts())
print("-"*50)
print(df["location_city"].value_counts())

single            1867
available           66
seeing someone      54
married             14
Name: status, dtype: int64
--------------------------------------------------
m    1139
f     862
Name: sex, dtype: int64
--------------------------------------------------
straight    1736
gay          153
bisexual     112
Name: orientation, dtype: int64
--------------------------------------------------
socially       1393
rarely          256
often           168
not at all      157
very often       16
desperately      11
Name: drinks, dtype: int64
--------------------------------------------------
never        1586
sometimes     397
often          18
Name: drugs, dtype: int64
--------------------------------------------------
no                1630
sometimes          128
yes                 94
when drinking       91
trying to quit      58
Name: smokes, dtype: int64
--------------------------------------------------
not interested         1050
interested              633
somewhat interested    

In [465]:
total_languages = []
total_language_proficiency = []
for ind in df.index:
    
    pets = df["pets"][ind]
            
    if "likes dogs" in pets:
        df.loc[ind,"likes_dogs"] = True
    elif "dislikes dogs" in pets:
        df.loc[ind,"likes_dogs"] = False
    else:
        df.loc[ind,"likes_dogs"] = False

    if "likes cats" in pets:
        df.loc[ind,"likes_cats"] = True
    elif "dislikes cats" in pets:
        df.loc[ind,"likes_cats"] = False
    else:
        df.loc[ind,"likes_cats"] = False

    if "has dogs" in pets:
        df.loc[ind,"has_dogs"] = True
    else:
        df.loc[ind,"has_dogs"] = False

    if "has cats" in pets:
        df.loc[ind,"has_cats"] = True
    else:
        df.loc[ind,"has_cats"] = False

    
    location = df["location"][ind]
    x = location.split(",")
    x[0] = x[0].strip()
    x[1] = x[1].strip()
    df.loc[ind,"location_state"] = x[1]
    df.loc[ind,"location_city"] = x[0]
    
    job = df["job"][ind]
    job = job.replace(" / ","/")
    df.loc[ind,"job"] = job
    
    language = df["language"][ind]
    language_json = []
    for lang in language.split(","):
        lang = lang.replace(" ","")
        lang = lang.replace(")","")
        lang = lang.strip()
        language_and_proficiency = lang.split("(")
        if len(language_and_proficiency) < 2:
            language_and_proficiency.append("fluently")
        if language_and_proficiency[0] not in total_languages:
            total_languages.append(language_and_proficiency[0])
        if language_and_proficiency[1] not in total_language_proficiency:
            total_language_proficiency.append(language_and_proficiency[1])
        lan_dict = {}
        lan_dict["language"] = language_and_proficiency[0]
        lan_dict["proficiency"] = language_and_proficiency[1]
        language_json.append(lan_dict)
    
    df.loc[ind,"language_json"] = json.dumps(language_json)
        
df.head()

,user_id,username,age,status,sex,orientation,drinks,drugs,height,job,location,pets,smokes,language,new_languages,body_profile,education_level,dropped_out,bio,interests,other_interests,location_preference,likes_dogs,likes_cats,has_cats,has_dogs,location_state,location_city,language_json
0,fffe3100,Edith Lopez,27,single,f,gay,socially,never,66.0,medicine/health,"oakland, california",likes dogs and likes cats,no,"english (fluently), spanish (poorly), sign lan...",interested,athletic,4.0,no,bottom line i love life! i work hard and i lov...,sports,instruments,same state,True,True,False,False,california,oakland,"[{""language"": ""english"", ""proficiency"": ""fluen..."
1,fffe3200,Travis Young,26,single,m,gay,socially,never,68.0,other,"pleasant hill, california",likes dogs,no,"english (fluently), tagalog (okay), french (po...",interested,fit,3.0,no,"i'm a straightforward, genuine, fun loving (i'...",painting,instruments,anywhere,True,False,False,False,california,pleasant hill,"[{""language"": ""english"", ""proficiency"": ""fluen..."
2,fffe3300,Agnes Smith,20,seeing someone,f,bisexual,socially,sometimes,69.0,other,"oakland, california",has dogs and likes cats,sometimes,"english (fluently), sign language (poorly), fr...",interested,fit,2.0,no,mmmmm yummy tacosss. yoga is where it's at. i ...,instruments,dancing,same city,False,True,False,True,california,oakland,"[{""language"": ""english"", ""proficiency"": ""fluen..."
3,fffe3400,Salvador Klaver,27,single,m,bisexual,socially,sometimes,68.0,computer/hardware/software,"daly city, california",likes dogs and likes cats,no,english,not interested,average,3.0,no,i'm a stealth geek. that special mix of techni...,sketching,acting,same city,True,True,False,False,california,daly city,"[{""language"": ""english"", ""proficiency"": ""fluen..."
4,fffe3500,Elana Sewell,22,single,f,bisexual,often,sometimes,68.0,other,"oakland, california",likes dogs and likes cats,yes,english,not interested,average,2.0,yes,with the whisper of the wind i was weaved into...,craft,designing,same city,True,True,False,False,california,oakland,"[{""language"": ""english"", ""proficiency"": ""fluen..."


In [466]:
lang_columns = []
for lang in total_languages:
    for prof in total_language_proficiency:
        lang_columns.append(lang +"_"+ prof)
df_language = pd.DataFrame(columns=lang_columns)

for ind in df.index:
    language_json = df["language_json"][ind]
    language_json = json.loads(language_json)
    for lan in language_json:
        column_name = lan["language"] + "_" + lan["proficiency"]
        df_language.loc[ind,column_name] = 1
    
df_language = df_language.fillna(0)
# df_language.head()
df = pd.concat([df,df_language],axis=1)
df.head()

,user_id,username,age,status,sex,orientation,drinks,drugs,height,job,location,pets,smokes,language,new_languages,body_profile,education_level,dropped_out,bio,interests,other_interests,location_preference,likes_dogs,likes_cats,has_cats,has_dogs,location_state,location_city,language_json,english_fluently,english_poorly,english_okay,spanish_fluently,spanish_poorly,spanish_okay,signlanguage_fluently,signlanguage_poorly,signlanguage_okay,tagalog_fluently,tagalog_poorly,tagalog_okay,french_fluently,french_poorly,french_okay,italian_fluently,italian_poorly,italian_okay,german_fluently,german_poorly,german_okay,chinese_fluently,chinese_poorly,chinese_okay,c++_fluently,c++_poorly,c++_okay,hebrew_fluently,hebrew_poorly,hebrew_okay,swedish_fluently,swedish_poorly,swedish_okay,hawaiian_fluently,hawaiian_poorly,hawaiian_okay,irish_fluently,irish_poorly,irish_okay,russian_fluently,russian_poorly,russian_okay,korean_fluently,korean_poorly,korean_okay,arabic_fluently,arabic_poorly,arabic_okay,portuguese_fluently,portuguese_poorly,portuguese_okay,japanese_fluently,japanese_poorly,japanese_okay,other_fluently,other_poorly,other_okay,thai_fluently,thai_poorly,thai_okay,esperanto_fluently,esperanto_poorly,esperanto_okay,croatian_fluently,croatian_poorly,croatian_okay,farsi_fluently,farsi_poorly,farsi_okay,greek_fluently,greek_poorly,greek_okay,ukrainian_fluently,ukrainian_poorly,ukrainian_okay,hindi_fluently,hindi_poorly,hindi_okay,sanskrit_fluently,sanskrit_poorly,sanskrit_okay,swahili_fluently,swahili_poorly,swahili_okay,icelandic_fluently,icelandic_poorly,icelandic_okay,bengali_fluently,bengali_poorly,bengali_okay,maori_fluently,maori_poorly,maori_okay,slovak_fluently,slovak_poorly,slovak_okay,norwegian_fluently,norwegian_poorly,norwegian_okay,latin_fluently,latin_poorly,latin_okay,vietnamese_fluently,vietnamese_poorly,vietnamese_okay,czech_fluently,czech_poorly,czech_okay,yiddish_fluently,yiddish_poorly,yiddish_okay,urdu_fluently,urdu_poorly,urdu_okay,turkish_fluently,turkish_poorly,turkish_okay,lisp_fluently,lisp_poorly,lisp_okay,indonesian_fluently,indonesian_poorly,indonesian_okay,slovenian_fluently,slovenian_poorly,slovenian_okay,polish_fluently,polish_poorly,polish_okay,danish_fluently,danish_poorly,danish_okay,ancientgreek_fluently,ancientgreek_poorly,ancientgreek_okay,afrikaans_fluently,afrikaans_poorly,afrikaans_okay,dutch_fluently,dutch_poorly,dutch_okay,albanian_fluently,albanian_poorly,albanian_okay,bulgarian_fluently,bulgarian_poorly,bulgarian_okay,finnish_fluently,finnish_poorly,finnish_okay,hungarian_fluently,hungarian_poorly,hungarian_okay,mongolian_fluently,mongolian_poorly,mongolian_okay,catalan_fluently,catalan_poorly,catalan_okay,tamil_fluently,tamil_poorly,tamil_okay,basque_fluently,basque_poorly,basque_okay,cebuano_fluently,cebuano_poorly,cebuano_okay,gujarati_fluently,gujarati_poorly,gujarati_okay,romanian_fluently,romanian_poorly,romanian_okay,tibetan_fluently,tibetan_poorly,tibetan_okay,armenian_fluently,armenian_poorly,armenian_okay,malay_fluently,malay_poorly,malay_okay,ilongo_fluently,ilongo_poorly,ilongo_okay,persian_fluently,persian_poorly,persian_okay
0,fffe3100,Edith Lopez,27,single,f,gay,socially,never,66.0,medicine/health,"oakland, california",likes dogs and likes cats,no,"english (fluently), spanish (poorly), sign lan...",interested,athletic,4.0,no,bottom line i love life! i work hard and i lov...,sports,instruments,same state,True,True,False,False,california,oakland,"[{""language"": ""english"", ""proficiency"": ""fluen...",1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,fffe3200,Travis Young,26,single,m,gay,socially,never,68.0,other,"pleasant hill, california",likes dogs,no,"englis

In [467]:
print(df["location_state"].value_counts())
print("-"*50)
print(df["location_city"].value_counts())

california    1997
vietnam          1
ohio             1
arizona          1
new york         1
Name: location_state, dtype: int64
--------------------------------------------------
san francisco     911
oakland           250
berkeley          146
san mateo          56
palo alto          45
                 ... 
brisbane            1
cincinnati          1
montara             1
point richmond      1
canyon country      1
Name: location_city, Length: 70, dtype: int64


In [468]:
label_object = {}
categorical_columns = [
    'status',
    'sex',
    'orientation',
    'drinks',
    'drugs',
    'smokes',
    'new_languages',
    'body_profile',
    'dropped_out',
    'interests',
    'other_interests',
    'location_preference',
    'likes_dogs',
    'likes_cats',
    'has_cats',
    'has_dogs',
    'location_state',
    'location_city'
]
for col in categorical_columns:
    labelencoder = LabelEncoder()
    labelencoder.fit(df[col])
    df[col] = labelencoder.fit_transform(df[col])
    label_object[col] = labelencoder
    
df.head()

,user_id,username,age,status,sex,orientation,drinks,drugs,height,job,location,pets,smokes,language,new_languages,body_profile,education_level,dropped_out,bio,interests,other_interests,location_preference,likes_dogs,likes_cats,has_cats,has_dogs,location_state,location_city,language_json,english_fluently,english_poorly,english_okay,spanish_fluently,spanish_poorly,spanish_okay,signlanguage_fluently,signlanguage_poorly,signlanguage_okay,tagalog_fluently,tagalog_poorly,tagalog_okay,french_fluently,french_poorly,french_okay,italian_fluently,italian_poorly,italian_okay,german_fluently,german_poorly,german_okay,chinese_fluently,chinese_poorly,chinese_okay,c++_fluently,c++_poorly,c++_okay,hebrew_fluently,hebrew_poorly,hebrew_okay,swedish_fluently,swedish_poorly,swedish_okay,hawaiian_fluently,hawaiian_poorly,hawaiian_okay,irish_fluently,irish_poorly,irish_okay,russian_fluently,russian_poorly,russian_okay,korean_fluently,korean_poorly,korean_okay,arabic_fluently,arabic_poorly,arabic_okay,portuguese_fluently,portuguese_poorly,portuguese_okay,japanese_fluently,japanese_poorly,japanese_okay,other_fluently,other_poorly,other_okay,thai_fluently,thai_poorly,thai_okay,esperanto_fluently,esperanto_poorly,esperanto_okay,croatian_fluently,croatian_poorly,croatian_okay,farsi_fluently,farsi_poorly,farsi_okay,greek_fluently,greek_poorly,greek_okay,ukrainian_fluently,ukrainian_poorly,ukrainian_okay,hindi_fluently,hindi_poorly,hindi_okay,sanskrit_fluently,sanskrit_poorly,sanskrit_okay,swahili_fluently,swahili_poorly,swahili_okay,icelandic_fluently,icelandic_poorly,icelandic_okay,bengali_fluently,bengali_poorly,bengali_okay,maori_fluently,maori_poorly,maori_okay,slovak_fluently,slovak_poorly,slovak_okay,norwegian_fluently,norwegian_poorly,norwegian_okay,latin_fluently,latin_poorly,latin_okay,vietnamese_fluently,vietnamese_poorly,vietnamese_okay,czech_fluently,czech_poorly,czech_okay,yiddish_fluently,yiddish_poorly,yiddish_okay,urdu_fluently,urdu_poorly,urdu_okay,turkish_fluently,turkish_poorly,turkish_okay,lisp_fluently,lisp_poorly,lisp_okay,indonesian_fluently,indonesian_poorly,indonesian_okay,slovenian_fluently,slovenian_poorly,slovenian_okay,polish_fluently,polish_poorly,polish_okay,danish_fluently,danish_poorly,danish_okay,ancientgreek_fluently,ancientgreek_poorly,ancientgreek_okay,afrikaans_fluently,afrikaans_poorly,afrikaans_okay,dutch_fluently,dutch_poorly,dutch_okay,albanian_fluently,albanian_poorly,albanian_okay,bulgarian_fluently,bulgarian_poorly,bulgarian_okay,finnish_fluently,finnish_poorly,finnish_okay,hungarian_fluently,hungarian_poorly,hungarian_okay,mongolian_fluently,mongolian_poorly,mongolian_okay,catalan_fluently,catalan_poorly,catalan_okay,tamil_fluently,tamil_poorly,tamil_okay,basque_fluently,basque_poorly,basque_okay,cebuano_fluently,cebuano_poorly,cebuano_okay,gujarati_fluently,gujarati_poorly,gujarati_okay,romanian_fluently,romanian_poorly,romanian_okay,tibetan_fluently,tibetan_poorly,tibetan_okay,armenian_fluently,armenian_poorly,armenian_okay,malay_fluently,malay_poorly,malay_okay,ilongo_fluently,ilongo_poorly,ilongo_okay,persian_fluently,persian_poorly,persian_okay
0,fffe3100,Edith Lopez,27,3,0,1,4,0,66.0,medicine/health,"oakland, california",likes dogs and likes cats,0,"english (fluently), spanish (poorly), sign lan...",0,1,4.0,0,bottom line i love life! i work hard and i lov...,25,13,2,1,1,0,0,1,40,"[{""language"": ""english"", ""proficiency"": ""fluen...",1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,fffe3200,Travis Young,26,3,1,1,4,0,68.0,other,"pleasant hill, california",likes dogs,0,"english (fluently), tagalog (okay), french (po...",0,4,3.0,0,"i'm a straightforward, genuine, fun loving (i'...

In [469]:
cleaned = df.job.str.split('/', expand=True).stack()
job_df = pd.get_dummies(cleaned,prefix="job").groupby(level=0).sum()
df = pd.concat([df,job_df],axis=1)
df.head()

,user_id,username,age,status,sex,orientation,drinks,drugs,height,job,location,pets,smokes,language,new_languages,body_profile,education_level,dropped_out,bio,interests,other_interests,location_preference,likes_dogs,likes_cats,has_cats,has_dogs,location_state,location_city,language_json,english_fluently,english_poorly,english_okay,spanish_fluently,spanish_poorly,spanish_okay,signlanguage_fluently,signlanguage_poorly,signlanguage_okay,tagalog_fluently,tagalog_poorly,tagalog_okay,french_fluently,french_poorly,french_okay,italian_fluently,italian_poorly,italian_okay,german_fluently,german_poorly,german_okay,chinese_fluently,chinese_poorly,chinese_okay,c++_fluently,c++_poorly,c++_okay,hebrew_fluently,hebrew_poorly,hebrew_okay,swedish_fluently,swedish_poorly,swedish_okay,hawaiian_fluently,hawaiian_poorly,hawaiian_okay,irish_fluently,irish_poorly,irish_okay,russian_fluently,russian_poorly,russian_okay,korean_fluently,korean_poorly,korean_okay,arabic_fluently,arabic_poorly,arabic_okay,portuguese_fluently,portuguese_poorly,portuguese_okay,japanese_fluently,japanese_poorly,japanese_okay,other_fluently,other_poorly,other_okay,thai_fluently,thai_poorly,thai_okay,esperanto_fluently,esperanto_poorly,esperanto_okay,croatian_fluently,croatian_poorly,croatian_okay,farsi_fluently,farsi_poorly,farsi_okay,greek_fluently,greek_poorly,greek_okay,ukrainian_fluently,ukrainian_poorly,ukrainian_okay,hindi_fluently,hindi_poorly,hindi_okay,sanskrit_fluently,sanskrit_poorly,sanskrit_okay,swahili_fluently,swahili_poorly,swahili_okay,icelandic_fluently,icelandic_poorly,icelandic_okay,bengali_fluently,bengali_poorly,bengali_okay,maori_fluently,maori_poorly,maori_okay,slovak_fluently,slovak_poorly,slovak_okay,norwegian_fluently,norwegian_poorly,norwegian_okay,latin_fluently,latin_poorly,latin_okay,vietnamese_fluently,vietnamese_poorly,vietnamese_okay,czech_fluently,czech_poorly,czech_okay,yiddish_fluently,yiddish_poorly,yiddish_okay,urdu_fluently,urdu_poorly,urdu_okay,turkish_fluently,turkish_poorly,turkish_okay,lisp_fluently,lisp_poorly,lisp_okay,indonesian_fluently,indonesian_poorly,indonesian_okay,slovenian_fluently,slovenian_poorly,slovenian_okay,polish_fluently,polish_poorly,polish_okay,danish_fluently,danish_poorly,danish_okay,ancientgreek_fluently,ancientgreek_poorly,ancientgreek_okay,afrikaans_fluently,afrikaans_poorly,afrikaans_okay,dutch_fluently,dutch_poorly,dutch_okay,albanian_fluently,albanian_poorly,albanian_okay,bulgarian_fluently,bulgarian_poorly,bulgarian_okay,finnish_fluently,finnish_poorly,finnish_okay,hungarian_fluently,hungarian_poorly,hungarian_okay,mongolian_fluently,mongolian_poorly,mongolian_okay,catalan_fluently,catalan_poorly,catalan_okay,tamil_fluently,tamil_poorly,tamil_okay,basque_fluently,basque_poorly,basque_okay,cebuano_fluently,cebuano_poorly,cebuano_okay,gujarati_fluently,gujarati_poorly,gujarati_okay,romanian_fluently,romanian_poorly,romanian_okay,tibetan_fluently,tibetan_poorly,tibetan_okay,armenian_fluently,armenian_poorly,armenian_okay,malay_fluently,malay_poorly,malay_okay,ilongo_fluently,ilongo_poorly,ilongo_okay,persian_fluently,persian_poorly,persian_okay,job_academia,job_administrative,job_artistic,job_banking,job_biz dev,job_clerical,job_computer,job_construction,job_craftsmanship,job_education,job_engineering,job_entertainment,job_executive,job_financial,job_government,job_hardware,job_health,job_hospitality,job_law,job_legal services,job_management,job_marketing,job_media,job_medicine,job_military,job_musical,job_other,job_political,job_rather not say,job_real estate,job_retired,job_sales,job_science,job_software,job_student,job_tech,job_transportation,job_travel,job_unemployed,job_writer
0,fffe3100,Edith Lopez,27,3,0,1,4,0,66.0,medicine/health,"oakland, california",likes dogs and likes cats,0,"english (fluently), spanish (poorly), sign lan...",0,1,4.0,0,bottom line i love life! i work hard and i lov...,25,13,2,1,1,0,0,1,40,"[{""language"": ""english"", ""proficiency"": ""fluen...",1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0

In [470]:
redundant_columns = ['username','job','location','pets','language','language_json']
df = df.drop(redundant_columns,axis=1)
df.head()

,user_id,age,status,sex,orientation,drinks,drugs,height,smokes,new_languages,body_profile,education_level,dropped_out,bio,interests,other_interests,location_preference,likes_dogs,likes_cats,has_cats,has_dogs,location_state,location_city,english_fluently,english_poorly,english_okay,spanish_fluently,spanish_poorly,spanish_okay,signlanguage_fluently,signlanguage_poorly,signlanguage_okay,tagalog_fluently,tagalog_poorly,tagalog_okay,french_fluently,french_poorly,french_okay,italian_fluently,italian_poorly,italian_okay,german_fluently,german_poorly,german_okay,chinese_fluently,chinese_poorly,chinese_okay,c++_fluently,c++_poorly,c++_okay,hebrew_fluently,hebrew_poorly,hebrew_okay,swedish_fluently,swedish_poorly,swedish_okay,hawaiian_fluently,hawaiian_poorly,hawaiian_okay,irish_fluently,irish_poorly,irish_okay,russian_fluently,russian_poorly,russian_okay,korean_fluently,korean_poorly,korean_okay,arabic_fluently,arabic_poorly,arabic_okay,portuguese_fluently,portuguese_poorly,portuguese_okay,japanese_fluently,japanese_poorly,japanese_okay,other_fluently,other_poorly,other_okay,thai_fluently,thai_poorly,thai_okay,esperanto_fluently,esperanto_poorly,esperanto_okay,croatian_fluently,croatian_poorly,croatian_okay,farsi_fluently,farsi_poorly,farsi_okay,greek_fluently,greek_poorly,greek_okay,ukrainian_fluently,ukrainian_poorly,ukrainian_okay,hindi_fluently,hindi_poorly,hindi_okay,sanskrit_fluently,sanskrit_poorly,sanskrit_okay,swahili_fluently,swahili_poorly,swahili_okay,icelandic_fluently,icelandic_poorly,icelandic_okay,bengali_fluently,bengali_poorly,bengali_okay,maori_fluently,maori_poorly,maori_okay,slovak_fluently,slovak_poorly,slovak_okay,norwegian_fluently,norwegian_poorly,norwegian_okay,latin_fluently,latin_poorly,latin_okay,vietnamese_fluently,vietnamese_poorly,vietnamese_okay,czech_fluently,czech_poorly,czech_okay,yiddish_fluently,yiddish_poorly,yiddish_okay,urdu_fluently,urdu_poorly,urdu_okay,turkish_fluently,turkish_poorly,turkish_okay,lisp_fluently,lisp_poorly,lisp_okay,indonesian_fluently,indonesian_poorly,indonesian_okay,slovenian_fluently,slovenian_poorly,slovenian_okay,polish_fluently,polish_poorly,polish_okay,danish_fluently,danish_poorly,danish_okay,ancientgreek_fluently,ancientgreek_poorly,ancientgreek_okay,afrikaans_fluently,afrikaans_poorly,afrikaans_okay,dutch_fluently,dutch_poorly,dutch_okay,albanian_fluently,albanian_poorly,albanian_okay,bulgarian_fluently,bulgarian_poorly,bulgarian_okay,finnish_fluently,finnish_poorly,finnish_okay,hungarian_fluently,hungarian_poorly,hungarian_okay,mongolian_fluently,mongolian_poorly,mongolian_okay,catalan_fluently,catalan_poorly,catalan_okay,tamil_fluently,tamil_poorly,tamil_okay,basque_fluently,basque_poorly,basque_okay,cebuano_fluently,cebuano_poorly,cebuano_okay,gujarati_fluently,gujarati_poorly,gujarati_okay,romanian_fluently,romanian_poorly,romanian_okay,tibetan_fluently,tibetan_poorly,tibetan_okay,armenian_fluently,armenian_poorly,armenian_okay,malay_fluently,malay_poorly,malay_okay,ilongo_fluently,ilongo_poorly,ilongo_okay,persian_fluently,persian_poorly,persian_okay,job_academia,job_administrative,job_artistic,job_banking,job_biz dev,job_clerical,job_computer,job_construction,job_craftsmanship,job_education,job_engineering,job_entertainment,job_executive,job_financial,job_government,job_hardware,job_health,job_hospitality,job_law,job_legal services,job_management,job_marketing,job_media,job_medicine,job_military,job_musical,job_other,job_political,job_rather not say,job_real estate,job_retired,job_sales,job_science,job_software,job_student,job_tech,job_transportation,job_travel,job_unemployed,job_writer
0,fffe3100,27,3,0,1,4,0,66.0,0,0,1,4.0,0,bottom line i love life! i work hard and i lov...,25,13,2,1,1,0,0,1,40,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [471]:
one_hot_object = {}
categorical_columns = [
    'status',
    'orientation',
    'drinks',
    'drugs',
    'smokes',
    'new_languages',
    'body_profile',
    'interests',
    'other_interests',
    'location_preference',
    'location_state',
    'location_city'
]
for col in categorical_columns:
    onehot_encoder = OneHotEncoder()
    col_data  = df[col].values.reshape(-1,1)
    onehot_encoder.fit(col_data)
    onehot_encoded = onehot_encoder.fit_transform(col_data).toarray()
    columns_list = []
    for i in range(onehot_encoded.shape[1]):
        columns_list.append(col+"_"+str(int(i)))
    dfOneHot = pd.DataFrame(onehot_encoded, columns = columns_list) 
    df = pd.concat([df, dfOneHot], axis=1)
    one_hot_object[col] = onehot_encoder
    
df = df.drop(categorical_columns,axis=1)
df.head()

,user_id,age,sex,height,education_level,dropped_out,bio,likes_dogs,likes_cats,has_cats,has_dogs,english_fluently,english_poorly,english_okay,spanish_fluently,spanish_poorly,spanish_okay,signlanguage_fluently,signlanguage_poorly,signlanguage_okay,tagalog_fluently,tagalog_poorly,tagalog_okay,french_fluently,french_poorly,french_okay,italian_fluently,italian_poorly,italian_okay,german_fluently,german_poorly,german_okay,chinese_fluently,chinese_poorly,chinese_okay,c++_fluently,c++_poorly,c++_okay,hebrew_fluently,hebrew_poorly,hebrew_okay,swedish_fluently,swedish_poorly,swedish_okay,hawaiian_fluently,hawaiian_poorly,hawaiian_okay,irish_fluently,irish_poorly,irish_okay,russian_fluently,russian_poorly,russian_okay,korean_fluently,korean_poorly,korean_okay,arabic_fluently,arabic_poorly,arabic_okay,portuguese_fluently,portuguese_poorly,portuguese_okay,japanese_fluently,japanese_poorly,japanese_okay,other_fluently,other_poorly,other_okay,thai_fluently,thai_poorly,thai_okay,esperanto_fluently,esperanto_poorly,esperanto_okay,croatian_fluently,croatian_poorly,croatian_okay,farsi_fluently,farsi_poorly,farsi_okay,greek_fluently,greek_poorly,greek_okay,ukrainian_fluently,ukrainian_poorly,ukrainian_okay,hindi_fluently,hindi_poorly,hindi_okay,sanskrit_fluently,sanskrit_poorly,sanskrit_okay,swahili_fluently,swahili_poorly,swahili_okay,icelandic_fluently,icelandic_poorly,icelandic_okay,bengali_fluently,bengali_poorly,bengali_okay,maori_fluently,maori_poorly,maori_okay,slovak_fluently,slovak_poorly,slovak_okay,norwegian_fluently,norwegian_poorly,norwegian_okay,latin_fluently,latin_poorly,latin_okay,vietnamese_fluently,vietnamese_poorly,vietnamese_okay,czech_fluently,czech_poorly,czech_okay,yiddish_fluently,yiddish_poorly,yiddish_okay,urdu_fluently,urdu_poorly,urdu_okay,turkish_fluently,turkish_poorly,turkish_okay,lisp_fluently,lisp_poorly,lisp_okay,indonesian_fluently,indonesian_poorly,indonesian_okay,slovenian_fluently,slovenian_poorly,slovenian_okay,polish_fluently,polish_poorly,polish_okay,danish_fluently,danish_poorly,danish_okay,ancientgreek_fluently,ancientgreek_poorly,ancientgreek_okay,afrikaans_fluently,afrikaans_poorly,afrikaans_okay,dutch_fluently,dutch_poorly,dutch_okay,albanian_fluently,albanian_poorly,albanian_okay,bulgarian_fluently,bulgarian_poorly,bulgarian_okay,finnish_fluently,finnish_poorly,finnish_okay,hungarian_fluently,hungarian_poorly,hungarian_okay,mongolian_fluently,mongolian_poorly,mongolian_okay,catalan_fluently,catalan_poorly,catalan_okay,tamil_fluently,tamil_poorly,tamil_okay,basque_fluently,basque_poorly,basque_okay,cebuano_fluently,cebuano_poorly,cebuano_okay,gujarati_fluently,gujarati_poorly,gujarati_okay,romanian_fluently,romanian_poorly,romanian_okay,tibetan_fluently,tibetan_poorly,tibetan_okay,armenian_fluently,armenian_poorly,armenian_okay,malay_fluently,malay_poorly,malay_okay,ilongo_fluently,ilongo_poorly,ilongo_okay,persian_fluently,persian_poorly,persian_okay,job_academia,job_administrative,job_artistic,job_banking,job_biz dev,job_clerical,job_computer,job_construction,job_craftsmanship,job_education,job_engineering,job_entertainment,job_executive,job_financial,job_government,job_hardware,job_health,job_hospitality,job_law,job_legal services,job_management,job_marketing,job_media,job_medicine,job_military,job_musical,job_other,job_political,job_rather not say,job_real estate,job_retired,job_sales,job_science,job_software,job_student,job_tech,job_transportation,job_travel,job_unemployed,job_writer,status_0,status_1,status_2,status_3,orientation_0,orientation_1,orientation_2,drinks_0,drinks_1,drinks_2,drinks_3,drinks_4,drinks_5,drugs_0,drugs_1,drugs_2,smokes_0,smokes_1,smokes_2,smokes_3,smokes_4,new_languages_0,new_languages_1,new_languages_2,body_profile_0,body_profile_1,body_profile_2,body_profile_3,body_profile_4,body_profile_5,body_profile_6,body_profile_7,body_profile_8,body_profile_9,body_profile_10,body_profile_11,interests_0,interests_1,interests_2,interests_3,interests_4,interests_5,interests_6,intere

In [472]:
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from unidecode import unidecode
import string
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

def pre_process(corpus):
    # convert input corpus to lower case.
    corpus = corpus.lower()
    # collecting a list of stop words from nltk and punctuation form
    # string class and create single array.
    stopset = stopwords.words('english') + list(string.punctuation)
    # remove stop words and punctuations from string.
    # word_tokenize is used to tokenize the input corpus in word tokens.
    corpus = " ".join([i for i in word_tokenize(corpus) if i not in stopset])
    # remove non-ascii characters
    corpus = unidecode(corpus)
    lemmatizer = WordNetLemmatizer()
    corpus = " ".join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(corpus)])
    return corpus

def my_pre_process(x,silent=True):

        if not silent:
            print("\n Printing First 5 rows \n")
            print(x[:5])

        x_2 = []
        all_tokens = []

        # nltk.download('stopwords')

        ps = PorterStemmer()
        stop_words = stopwords.words('english')
        tokenizer = RegexpTokenizer(r'\w+')

        # Removing Punctuations
        for desc in x:
            # print(desc)
            all_tokens.append(tokenizer.tokenize(desc))

        # print(all_tokens)

        if not silent:
            print("\n Pre-Processing Text \n")
        # PRE-PROCESSING TEXT

        # all_tokens contains list of list of descriptions like [[desc1_w1,desc1_w2],[desc2_w1,desc2_w2]...]
        for desc_tokens in all_tokens:
            # desc_tokens have first description sentence in list of tokens of words
            stemmed_desc_tokens = []
            for tok in desc_tokens:
                # tok is the first token word of from desc_tokens
                tok = tok.lower()
                # converted into lower case
                if tok not in stop_words:
                    # if it is not in stop_words (stop_words is set having only .,\! such punctuations)
                    # so if it is not in stop_words that means its english word so all punctuations will be skipped
                    stemmed_word = ps.stem(tok)
                    # it is now done stemmed so root of that tok is stemmed_word
                    # Now removing single character tokens which are totally useless for training
                    if len(stemmed_word) > 1:
                        # Now checking if the word which is stemmed is alphabetic only
                        # then only add to the final list
                        if stemmed_word.isalpha():
                            stemmed_desc_tokens.append(stemmed_word)

            x_2.append(stemmed_desc_tokens)

        # print(x_2[:5])

        for index, desc_tokens in enumerate(x_2):
            desc = " ".join(tok for tok in desc_tokens)
            x_2[index] = desc

        if not silent:
            print("\n Printing First 5 rows AFTER PRE-PROCESSING : \n")
            print(x_2[:5])

        # Free memory
        x = x_2
        x_2 = None

        return x

In [473]:
df["bio"] =  df.apply(lambda row : my_pre_process([row["bio"]])[0],axis=1)

In [474]:
index_values = list(df['user_id'])
df[["user_id","bio"]].head()

,user_id,bio
0,fffe3100,bottom line love life work hard love hard eart...
1,fffe3200,straightforward genuin fun love seriou profil ...
2,fffe3300,mmmmm yummi tacosss yoga love life part conten...
3,fffe3400,stealth geek special mix technic obsess enough...
4,fffe3500,whisper wind weav curl crash wave tide spit on...


In [475]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2))
tfidf_vectorizer.fit(df["bio"])
feature_vectors = tfidf_vectorizer.transform(df["bio"])

In [476]:
df = pd.concat([df,pd.DataFrame(feature_vectors.toarray())],axis=1)
df = df.drop(["bio"],axis=1)

In [477]:
df = df.drop(["user_id"],axis=1)

from sklearn.preprocessing import StandardScaler

df = StandardScaler().fit_transform(df)

In [478]:
pca_components_per = 0.06
from sklearn.decomposition import PCA
pca = PCA(pca_components_per)
pca = pca.fit(df)
df = pca.transform(df)

In [479]:
df = pd.DataFrame(df)
df.shape

(2001, 17)

In [480]:
u_u_similarities = cosine_similarity(df)
u_u_similarities = u_u_similarities*100

df_to_submit = pd.DataFrame(data = u_u_similarities,  
                  index = index_values,  
                  columns =index_values)
df_to_submit.index.name = "user_id"
df_to_submit.head()

fffe3100    fffe3200    fffe3300    fffe3400    fffe3500  \
user_id                                                                
fffe3100  100.000000  -25.317914   76.053500   76.570756   82.122642   
fffe3200  -25.317914  100.000000   -7.197038  -18.004408  -37.864873   
fffe3300   76.053500   -7.197038  100.000000   63.154458   64.880583   
fffe3400   76.570756  -18.004408   63.154458  100.000000   76.890542   
fffe3500   82.122642  -37.864873   64.880583   76.890542  100.000000   

           fffe3600   fffe3700   fffe3800   fffe3900  fffe31003000  \
user_id                                                              
fffe3100  82.548124  87.342263  82.539233  82.429565     72.602449   
fffe3200 -31.843970 -29.035676 -36.526174 -29.875531    -31.209615   
fffe3300  68.783388  65.860935  64.263230  66.363157     77.789238   
fffe3400  90.506958  88.553810  83.195386  88.773247     83.573258   
fffe3500  91.247035  93.116573  92.272690  93.930822     83.693248   

          fffe31003100  fffe31003200  fffe31003300  fffe31003400  \
user_id                                                            
fffe3100     78.102329     75.085905     82.912872     81.852330   
fffe3200    -28.712140    -38.311642    -32.899274    -25.384305   
fffe3300     61.382048     53.331088     67.060811     64.904902   
fffe3400     90.456624     74.898200     85.230732     88.461128   
fffe3500     87.747429     88.193169     93.894705     92.349593   

          fffe31003500  fffe31003600  fffe31003700  fffe31003800  \
user_id                                                            
fffe3100     83.851352     79.780373     83.381481     80.444310   
fffe3200    -33.574480    -24.935970    -27.709614    -30.568755   
fffe3300     65.479965     64.103358     75.493335     64.420113   
fffe3400     87.298614     87.612510     77.991078     86.674508   
fffe3500     92.738475     88.374042     87.601593     85.034296   

          fffe31003900  fffe32003000  fffe32003100  fffe32003200  \
user_id                                                            
fffe3100     73.031898     82.117121     57.662419     44.476239   
fffe3200    -38.215407    -28.636965    -39.985752    -17.247541   
fffe3300     63.917569     65.945644     42.601255     25.130338   
fffe3400     70.431276     84.299129     52.944914     67.574657   
fffe3500     93.437779     93.415673     71.383578     36.328546   

          fffe32003300  fffe32003400  fffe32003500  fffe32003600  \
user_id                                                            
fffe3100     80.550940     82.977740     71.377032     84.330376   
fffe3200    -33.343496    -34.666750    -22.123016    -30.931210   
fffe3300     72.367280     65.231098     56.553130     73.292164   
fffe3400     85.743062     85.482616     70.260422     85.218320   
fffe3500     90.590875     94.366135     81.905338     92.649290   

          fffe32003700  fffe32003800  fffe32003900  fffe33003000  \
user_id                                                            
fffe3100     81.460383     81.939661     73.592768     75.384032   
fffe3200    -31.184095    -37.326040    -16.461478    -30.928751   
fffe3300     56.990758     62.060806     45.205347     58.705365   
fffe3400     78.512085     89.406247     61.822340     76.649856   
fffe3500     84.713132     92.189887     65.595661     90.731200   

          fffe33003100  fffe33003200  fffe33003300  fffe33003400  \
user_id                                                            
fffe3100     80.890640     84.810910     79.629379     47.612645   
fffe3200    -37.163119    -25.710731    -31.013930    -32.600583   
fffe3300     63.259269     68.609339     63.877268     58.059879   
fffe3400     78.992942     88.350547     83.789003     50.047871   
fffe3500     96.062685     90.590740     91.480382     54.937786   

          fffe33003500  fffe33003600  fffe33003700  fffe33003800  \
user_id                                                            
fffe3100     82.34

In [481]:
for x in index_values:
    df_to_submit.loc[x,x] = 0
df_to_submit[df_to_submit < 0] = 0
df_to_submit[df_to_submit > 100] = 100

In [482]:
df = pd.read_csv(filepath_or_buffer="./dataset/data.csv")

df["location_state"] = None
df["location_city"] = None

for ind in df.index:
    
    location = df["location"][ind]
    x = location.split(",")
    x[0] = x[0].strip()
    x[1] = x[1].strip()
    df.loc[ind,"location_state"] = x[1]
    df.loc[ind,"location_city"] = x[0]

df.set_index("user_id", inplace=True)
df.head()

,username,age,status,sex,orientation,drinks,drugs,height,job,location,pets,smokes,language,new_languages,body_profile,education_level,dropped_out,bio,interests,other_interests,location_preference,location_state,location_city
user_id,,,,,,,,,,,,,,,,,,,,,,,
fffe3100,Edith Lopez,27,single,f,gay,socially,never,66.0,medicine / health,"oakland, california",likes dogs and likes cats,no,"english (fluently), spanish (poorly), sign lan...",interested,athletic,4.0,no,bottom line i love life! i work hard and i lov...,sports,instruments,same state,california,oakland
fffe3200,Travis Young,26,single,m,gay,socially,never,68.0,other,"pleasant hill, california",likes dogs,no,"english (fluently), tagalog (okay), french (po...",interested,fit,3.0,no,"i'm a straightforward, genuine, fun loving (i'...",painting,instruments,anywhere,california,pleasant hill
fffe3300,Agnes Smith,20,seeing someone,f,bisexual,socially,sometimes,69.0,other,"oakland, california",has dogs and likes cats,sometimes,"english (fluently), sign language (poorly), fr...",interested,fit,2.0,no,mmmmm yummy tacosss. yoga is where it's at. i ...,instruments,dancing,same city,california,oakland
fffe3400,Salvador Klaver,27,single,m,bisexual,socially,sometimes,68.0,computer / hardware / software,"daly city, california",likes dogs and likes cats,no,english,not interested,average,3.0,no,i'm a stealth geek. that special mix of techni...,sketching,acting,same city,california,daly city
fffe3500,Elana Sewell,22,single,f,bisexual,often,sometimes,68.0,other,"oakland, california",likes dogs and likes cats,yes,english,not interested,average,2.0,yes,with the whisper of the wind i was weaved into...,craft,designing,same city,california,oakland


In [483]:
for ind in df_to_submit.index:
    my_info = df.loc[ind]
    my_gender = my_info["sex"]
    my_orientation = my_info["orientation"]
    if my_orientation == "straight":
        tmp = df.loc[(df["orientation"] == "straight") & (df["sex"] == my_gender)]
        tmp_index = list(tmp.index)
        df_to_submit.loc[ind,tmp_index] = 0
    elif my_orientation == "gay":
        tmp = df.loc[(df["orientation"] == "gay") & (df["sex"] != my_gender)]
        tmp_index = list(tmp.index)
        df_to_submit.loc[ind,tmp_index] = 0
    
    my_location_preference = my_info["location_preference"]
    my_state = my_info["location_state"]
    my_city = my_info["location_city"]
    if my_location_preference == "same city":
        tmp = df.loc[df["location_city"] != my_city]
        tmp_index = list(tmp.index)
        df_to_submit.loc[ind,tmp_index] = 0
    elif my_location_preference == "same state":
        tmp = df.loc[df["location_state"] != my_state]
        tmp_index = list(tmp.index)
        df_to_submit.loc[ind,tmp_index] = 0    

In [484]:
df_to_submit.to_csv(path_or_buf="./df_to_submit_location_"+str(pca_components_per*100)+".csv",float_format='%.1f')